In [36]:
import numpy as np
from scipy import misc
from skimage.util import view_as_windows as viewW

In [69]:
def im2col(img, size, stepsize=1):
    A = np.lib.pad(img, ((0,((size-len(img))%size)), (0,(size-len(img[0]))%size)), "constant", constant_values=0)
    # Parameters
    m,n = A.shape
    s0, s1 = A.strides    
    nrows = m-size+1
    ncols = n-size+1
    shp = size,size,nrows,ncols
    strd = s0,s1,s0,s1

    out_view = np.lib.stride_tricks.as_strided(A, shape=shp, strides=strd)
    return out_view.reshape(size*size,-1)[:,::stepsize]

In [114]:
def convolve(img, filter):
    newImg = im2col(img, len(filter))
    print(len(filter))
    print(newImg)
    newImg = np.dot(newImg, filter)
    return newImg.reshape((len(img),len(img[0])))
def convolveLayer(layer, filters, newLayerNum):
    layerNum = len(layer)
    layerSize = [len(layer[0]),len(layer[0][0])]
    outputLayer = np.zeros((newLayerNum, layerSize[0],layerSize[1]))
    for j in range(newLayerNum):
        for i in range(layerNum):
            outputLayer[i]=np.add(outputLayer[i],convolve(layer[j], filters[i][j]))
    return outputLayer
def maxPool(img, size):
    newImg = im2col(img, size,stepsize=size)
    newImg = np.amax(newImg, axis=1)
    return newImg.reshape((len(img),len(img[0])))
def poolLayer(layer, size):
    layerNum = len(layer)
    layerSize = [len(layer[0]),len(layer[0][0])]
    outputLayer = [None]*layerNum
    for i in range(layerNum):
        outputLayer[i] = maxPool(layer[i], size)
    return outputLayer
def reluLayer(layer, leak=0.01):
    gradients = 1. * (layer > 0)
    gradients[gradients == 0] = leak
    return gradients

In [115]:
img = np.divide(misc.imread("/Users/frank/Desktop/Github/Flag-Recognizer/save_save_save_america08.jpg", flatten=True),255)-0.5

In [116]:
def initFilters(layerSizes, size):
    filters = [None]*(len(layerSizes)-1)
    for i in range(len(layerSizes)-1):
        filters[i] = 0.01*np.random.random((layerSizes[i],layerSizes[i+1], size*size))-0.005
    return filters

In [117]:
layerSizes=[1, 10, 10, 4]
filters=initFilters(layerSizes, 5)
layer1 = convolveLayer([img], filters[0], layerSizes[1])
layer2 = convolveLayer(layer1, filters[1], layerSizes[2])
layer3 = convolveLayer(layer2, filters[2], layerSizes[3])
print(layer3)

25
[[-0.45831764]
 [-0.48075685]
 [-0.4844    ]
 [-0.4637059 ]
 [-0.49539608]
 [-0.4965255 ]
 [-0.47685099]
 [-0.49309412]
 [-0.48384315]
 [-0.48849019]
 [-0.4416196 ]
 [-0.49079216]
 [-0.47745097]
 [-0.48851374]
 [-0.49703923]
 [-0.47532156]
 [-0.47224706]
 [-0.47889411]
 [-0.47303137]
 [-0.47537649]
 [-0.47654903]
 [-0.47303137]
 [-0.48475686]
 [-0.49179214]
 [-0.49535295]
 [-0.48446667]
 [-0.44056469]
 [-0.45733726]
 [-0.49546275]
 [-0.45624706]
 [-0.4951843 ]
 [-0.46862745]
 [-0.49769804]
 [-0.49539608]
 [-0.49539608]
 [-0.49539608]
 [-0.49882746]
 [-0.4906196 ]
 [-0.43575686]
 [-0.47889411]
 [-0.47654903]
 [-0.47654903]
 [-0.47537649]
 [-0.4572157 ]
 [-0.46834117]
 [-0.4544    ]
 [-0.47654903]
 [-0.46959999]
 [-0.48944706]
 [-0.49648234]
 [ 0.49652547]
 [-0.47592157]
 [-0.44433725]
 [-0.45750982]
 [-0.33807451]
 [-0.41723138]
 [-0.38590196]
 [-0.33609411]
 [-0.32336861]
 [-0.29855686]
 [-0.20978433]
 [-0.13867843]
 [-0.1302745 ]
 [-0.1661804 ]
 [-0.18303531]
 [-0.13145491]
 [-0.21

ValueError: shapes (625,1) and (25,) not aligned: 1 (dim 1) != 25 (dim 0)

In [96]:
#test im2col
print(im2col(np.array([[6,5,4],[1,2,3]]),2,stepsize=2))

[[6 4]
 [5 0]
 [1 3]
 [2 0]]
